# Raasch's Hook with linear HHJ formulation
In this notebook we consider Raasch's Hook, which is clamped on the left a shear force is applied on the right and the other two boundaries are left free. 

The material and geometrical properties are
$$
E = 3300,\qquad \nu = 0.35,\qquad R_1=14,\qquad R_2=46,\qquad w=20, \qquad t\in\{20,2,0.02,0.02\}
$$
the shear force is $P=(t/2)^3/w$, and we consider the HHJ formulation for linear Kirchhoff-Love shells:
\begin{align*}
\mathcal{L}(u,\boldsymbol{m})&=\int_S \frac{t}{2}\|\mathcal{I}^{\mathrm{Reg}}\mathrm{sym}(P_S\nabla_S u)\|_{\mathbb{M}}^2  -\frac{6}{t^3}\|\boldsymbol{m}\|^2_{\mathbb{M}^{-1}} + \sum_{T\in\mathcal{T}} \int_T\boldsymbol{m}: \mathcal{H}_{\hat{\nu}}\,dx \\
&\qquad-\int_{\partial T}\boldsymbol{m}_{\hat\mu\hat\mu}(\nabla_S u^T)_{\hat\nu\hat\mu}\,ds - \int_{\Gamma_{\mathrm{right}}}f\cdot u\,ds
\end{align*}
with the surface Hessian $\mathcal{H}_{\hat{\nu}} := \sum_{i}\nabla^2_Su_i {\hat{\nu}}_i$, unit normal vector ${\hat{\nu}}$, edge-tangential vector $\hat\tau$, and the co-normal (element-normal) vector $\hat\mu=\hat\nu\times \hat\tau$.

In [ ]:
# Load NGSolve module, and the Open Cascade (OCC) module
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import math

# Geometry and material parameters
R1 = 14
R2 = 46
w = 20
E, nu = 3300, 0.35


# Create geometry and mesh
def GenerateMesh(maxh=5):
    M1 = [0, R1]
    M2 = [
        M1[0] + (R1 + R2) * math.cos(30 / 180 * pi),
        M1[1] + -(R1 + R2) * math.sin(30 / 180 * pi),
    ]

    X0 = 0
    Y0 = 60 * math.sin(pi / 6)

    p1 = gp_Pnt(X0 + R1 * math.sin(0), Y0 - R1 * math.cos(0), 0)
    p2 = gp_Pnt(X0 + R1 * math.sin(pi / 6), Y0 - R1 * math.cos(pi / 6), 0)
    p3 = gp_Pnt(X0 + R1 * math.sin(pi / 3), Y0 - R1 * math.cos(pi / 3), 0)

    arc1 = ArcOfCircle(p1, p2, p3)

    angle1 = -pi / 3 + 5 * pi / 6 * 0.5
    angle2 = -pi / 3 + 5 * pi / 6
    X0 = (R1 + R2) * math.cos(pi / 6)
    Y0 = 0
    p4 = gp_Pnt(X0 + R2 * math.sin(angle1), Y0 + R2 * math.cos(angle1), 0)
    p5 = gp_Pnt(X0 + R2 * math.sin(angle2), Y0 + R2 * math.cos(angle2), 0)

    arc2 = ArcOfCircle(p3, p4, p5)

    wire = Wire([arc1, arc2])
    shell = Prism(wire, (0, 0, w))

    point_P = (X0 + R2 * math.sin(angle2), Y0 + R2 * math.cos(angle2), w / 2)
    v1 = Vertex(point_P)
    v1.name = "poi"
    shape = Glue([shell, v1])

    shape.edges.name = "side"
    for e in shape.SubShapes(EDGE):
        rho, c = e.Properties()
        if c.x < 0.01:
            e.name = "left"
        if abs(c.x - R1 * math.cos(30 / 180 * pi)) < 0.1:
            e.name = "int"
        if c.x > M2[0] + R2 - 0.01:
            e.name = "right"

    return Mesh(OCCGeometry(shape).GenerateMesh(maxh=maxh)), point_P

At the clamped boundary we need to fix the displacement by homogeneous Dirichlet boundary conditions and the hybridization variable, which has the physical meaning of the rotated angle.

In [ ]:
# Solve Kirchhoff-Love shell equations with given mesh, thickness, and polynomial order
def Solve(mesh, thickness=20, order=4):
    mesh.Curve(order)

    force = CF((0, 0, -((thickness / 2) ** 3) / w))

    fesU = VectorH1(mesh, order=order, dirichlet_bbnd="left")
    fesM = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
    fesH = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="left")
    fes = fesU * fesM * fesH

    (u, m, hyb), (du, dm, dhyb) = fes.TnT()
    m, hyb, dm, dhyb = m.Trace(), hyb.Trace(), dm.Trace(), dhyb.Trace()

    Regge = HCurlCurl(mesh, order=order - 1, discontinuous=True)

    gf_solution = GridFunction(fes, name="solution")

    nv = specialcf.normal(mesh.dim)
    tv = specialcf.tangential(mesh.dim)
    cnv = Cross(nv, tv)

    Ptau = Id(mesh.dim) - OuterProduct(nv, nv)

    def MaterialStress(mat, E, nu):
        return E / (1 - nu**2) * ((1 - nu) * mat + nu * Trace(mat) * Ptau)

    def MaterialStressInv(mat, E, nu):
        return (1 + nu) / E * (mat - nu / (nu + 1) * Trace(mat) * Ptau)

    Hn = (u.Operator("hesseboundary").trans * nv).Reshape((3, 3))
    dHn = (du.Operator("hesseboundary").trans * nv).Reshape((3, 3))

    a = BilinearForm(fes, symmetric=True, condense=True)
    a += (
        thickness
        * InnerProduct(
            MaterialStress(Interpolate(Sym(Ptau * Grad(u).Trace()), Regge), E, nu),
            Interpolate(Sym(Ptau * Grad(du).Trace()), Regge),
        )
    ).Compile() * ds
    a += (
        -12 / thickness**3 * InnerProduct(MaterialStressInv(m, E, nu), dm)
    ).Compile() * ds
    a += (InnerProduct(dm, Hn) + InnerProduct(m, dHn)).Compile() * ds
    a += (
        -m[cnv, cnv] * (Grad(du).Trace()[nv, cnv] - dhyb * cnv)
        - dm[cnv, cnv] * (Grad(u).Trace()[nv, cnv] - hyb * cnv)
    ).Compile() * ds(element_boundary=True)

    f = LinearForm(fes)
    f += InnerProduct(force, du).Compile() * dx(definedon=mesh.BBoundaries("right"))

    with TaskManager():
        a.Assemble()
        f.Assemble()
        r = f.vec.CreateVector()
        if a.condense:
            r.data = f.vec
            r.data += a.harmonic_extension_trans * r
            inv = a.mat.Inverse(fes.FreeDofs(True), inverse="sparsecholesky")
            gf_solution.vec.data = solvers.PreconditionedRichardson(
                a, r, inv, maxit=3, tol=1e-13, printing=False
            )
            gf_solution.vec.data += a.harmonic_extension * gf_solution.vec
            gf_solution.vec.data += a.inner_solve * r
        else:
            inv = a.mat.Inverse(fes.FreeDofs(), inverse="umfpack")
            r.data = f.vec
            gf_solution.vec.data = solvers.PreconditionedRichardson(
                a, r, inv, maxit=3, tol=1e-13, printing=False
            )

    return gf_solution

In [ ]:
# Mesh and point of interest
mesh, point_P = GenerateMesh(maxh=0.5)

# List of thicknesses
thicknesses = [20, 2, 0.2, 0.02]

# Store results
result_P = []

for t in thicknesses:
    gf_solution = Solve(mesh, thickness=t, order=4)
    gfu, gfm, _ = gf_solution.components

    result_P.append((t, abs(gfu(mesh(*point_P, BND))[2])))

# Print results
print(f"Deflections = {result_P}")